This jupyter notebook is built with these previous contributors:
1. https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d Keras Model for Beginners (0.210 on LB)+EDA+R&D by DeveshMaheshwari
2. https://www.kaggle.com/toregil/welcome-to-deep-learning-cnn-99 Welcome to deep learning (CNN 99%) by Peter Grenholm
3. https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/ Display Deep Learning Model Training History in Keras by Jason Brownlee
4. https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/ Evaluate the Performance Of Deep Learning Models in Keras by Jason Brownlee

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
from scipy.ndimage.filters import median_filter
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline


# data processing

### Import the data

In [2]:
#Load the data.
#train = pd.read_json("train.json")
#test = pd.read_json("test.json")

In [3]:
#train.head()

In [4]:
#X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
#X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

In [5]:
#np.save('trainX_1',X_band_1)
#np.save('trainX_1',X_band_1)

In [6]:
#np.load('trainX_1.npy')

In [7]:
#plt.imshow(X_band_1[1])
#X_band_1.shape

In [8]:
#X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis]], axis=-1)

### Rotate the image 90, 180 degrees 

In [9]:
'''X_band_1_rotate90 = np.array([np.rot90(image) for image in X_band_1])
X_band_2_rotate90 = np.array([np.rot90(image) for image in X_band_2])
plt.imshow(X_band_1_rotate90[1])

X_band_1_rotate180 = np.array([np.rot90(image,2) for image in X_band_1])
X_band_2_rotate180 = np.array([np.rot90(image,2) for image in X_band_2])
plt.figure()
plt.imshow(X_band_1_rotate180[1])'''

'X_band_1_rotate90 = np.array([np.rot90(image) for image in X_band_1])\nX_band_2_rotate90 = np.array([np.rot90(image) for image in X_band_2])\nplt.imshow(X_band_1_rotate90[1])\n\nX_band_1_rotate180 = np.array([np.rot90(image,2) for image in X_band_1])\nX_band_2_rotate180 = np.array([np.rot90(image,2) for image in X_band_2])\nplt.figure()\nplt.imshow(X_band_1_rotate180[1])'

In [10]:
#X_train_rotate90 = np.concatenate([X_band_1_rotate90[:, :, :, np.newaxis], X_band_2_rotate90[:, :, :, np.newaxis]], axis=-1)
#X_train_rotate180 = np.concatenate([X_band_1_rotate180[:, :, :, np.newaxis], X_band_2_rotate180[:, :, :, np.newaxis]], axis=-1)

In [11]:
#X_train_rotate = np.concatenate([X_train, X_train_rotate90, X_train_rotate180],axis = 0)
#target_train=np.concatenate([train['is_iceberg'],train['is_iceberg'],train['is_iceberg']],axis = 0)


In [12]:
#np.save('X_train_rotate',X_train_rotate)
#np.save('target_train',target_train)

In [13]:
X_train_rotate = np.load("X_train_rotate.npy")
target_train = np.load("target_train.npy")

for i in range(100):
    fig = plt.figure()
    ax = fig.add_subplot(1,2,1)
    ax.imshow(X_band_1[i])
    ax2= fig.add_subplot(1,2,2)
    ax2.imshow(X_band_2[i])
    plt.title(train['is_iceberg'][i])
    

# Convnet with Keras

### keras init

In [14]:
import keras
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, BatchNormalization
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 2)))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]


file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

### Split data for validation

In [ ]:
import timeit
start = timeit.default_timer()

In [ ]:
'''# fix random seed for reproducibility

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []

import os
for train_index, test_index in kfold.split(X_train_rotate, target_train):
    gmodel=getModel()
    history = gmodel.fit(X_train_rotate[train_index], target_train[train_index], 
              batch_size = 24,
              epoches = 10,
              verbose = 1)
    scores = gmodel.evaluate(X_train_rotate[test_index], target_train[test_index])
    cvscores.append(scores[1]*100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores))) 

'''

In [ ]:
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train_rotate, target_train, random_state=1, train_size=0.9)
#Without denoising, core features.

import os
gmodel=getModel()
history = gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=30,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=None)

In [ ]:
stop = timeit.default_timer()
print (stop-start)

### Training with Keras

In [ ]:
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### plot training history

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Save the weights for later use (optional)

In [ ]:
#gmodel.save('weights.h5')

# predict the test.json 

In [ ]:
#X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
#X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
#X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
#                          , X_band_test_2[:, :, :, np.newaxis]], axis=-1)
X_test = np.load("X_test.npy")
#predicted_test=gmodel.predict_proba(X_test)

In [ ]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)